In [1]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import itertools
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp1d

import multiprocessing as mp
from rich import print, progress
import psutil
import copy
import pickle

In [2]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times"] + plt.rcParams["font.serif"]
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['mathtext.rm'] = 'Serif'
mpl.rcParams['mathtext.it'] = 'Serif'
mpl.rcParams['mathtext.bf'] = 'Serif'
mpl.rcParams['text.usetex']= False
plt.rc('axes', linewidth=2)
plt.rc('font', weight='bold')

In [3]:
!pwd

/home/u1159830/workspace/MESA-tests


Get grid data

In [5]:
def get_data(index, grid_dir):
    prof_index_i = np.loadtxt(f"{grid_dir}/profiles/profiles_{index}.index", skiprows=1, dtype=int)
    freqs_i = []
    n_profs_i = []
    freq_files = sorted(glob.glob(f"{grid_dir}/gyre/freqs_track_{index}/profile*-freqs.dat"), key=lambda x: int(os.path.basename(x).split('.')[0].split('profile')[1].split('-')[0]))
    for file in freq_files:
        freqs_i.append(pd.read_table(file, skiprows=5, sep='\s+'))
        n_profs_i.append(int(file.split('profile')[-1].split('-')[0]))
    hist_i = pd.read_table(f"{grid_dir}/histories/history_{index}.data", skiprows=5, sep='\s+')
    return hist_i, freqs_i, n_profs_i, prof_index_i


def get_all(grid_dir="../MESA-grid/grid_urot"):
    hists = []
    freqs = []
    n_profs = []
    prof_indexes = []

    with progress.Progress() as pbar:
        task = pbar.add_task("[green]Reading data...", total=14872)
        for i in range(1, 14873):
            try:
                hist_i, freqs_i, n_profs_i, prof_index_i = get_data(i, grid_dir=grid_dir)
                hists.append(hist_i)
                freqs.append(freqs_i)
                n_profs.append(n_profs_i)
                prof_indexes.append(prof_index_i)
            except:
                print(f"Failed to get data for grid index {i}")
            pbar.advance(task)
    return hists, freqs, n_profs, prof_indexes


def get_data_parallel(args):
    index, grid_dir, hists, freqs, n_profs, prof_indexes = args
    try:
        hist_i, freqs_i, n_profs_i, prof_index_i = get_data(index, grid_dir=grid_dir)
        hists.append(hist_i)
        freqs.append(freqs_i)
        n_profs.append(n_profs_i)
        prof_indexes.append(prof_index_i)
    except:
        print(f"Failed to get data for grid index {index}")
        
        
def get_all_parallel(grid_dir="../MESA-grid/grid_urot"):
    manager = mp.Manager()
    hists = manager.list()
    freqs = manager.list()
    n_profs = manager.list()
    prof_indexes = manager.list()
    
    with progress.Progress() as pbar:
        task = pbar.add_task("[green]Reading data...", total=14872)
        with mp.Pool(psutil.cpu_count(logical=False)) as pool:
            args = [(i, grid_dir, hists, freqs, n_profs, prof_indexes) for i in range(1, 14873)]
            for res in pool.imap_unordered(get_data_parallel, args):
                pbar.advance(task)
    return list(hists), list(freqs), list(n_profs), list(prof_indexes)


In [7]:
# hists, freqs, n_profs, prof_indexes = get_all("../MESA-grid/grid_urot")

hists_all, freqs_all, n_profs_all, prof_indexes_all = get_all_parallel("../MESA-grid/grid_urot")

Output()

In [18]:
len(hists_all[:])

14872

In [16]:
n_profs_all

<ListProxy object, typeid 'list' at 0x151df71b1400>

#### Read from pickle file

In [4]:
with open('data/hists.pkl', 'rb') as f:
    hists = pickle.load(f)
with open('data/freqs.pkl', 'rb') as f:
    freqs = pickle.load(f)
with open('data/n_profs.pkl', 'rb') as f:
    n_profs = pickle.load(f)
with open('data/prof_indexes.pkl', 'rb') as f:
    prof_indexes = pickle.load(f)

$\Delta \nu$ and $\epsilon$

In [5]:
def fit_radial(ts, degree=0):
    """
    Fits a straight line to the radial mode frequencies. Optionally, can be used on non-radial modes.
    Only modes with radial orders 5-9 are used, as the ridges should be vertical here.
    
    Input: Theoretical (or observed) spectrum in pandas df format; mode degree to be used (default 0 = radial)
    Output: The length of the series used, and the slope, intercept, r_value, p_value, and std_err of the line.
    """
    n_min, n_max = 5, 9
    try:
        vert_freqs = ts.query("n_g == 0").query(f"l=={degree}").query(f"n_pg>={n_min}").query(f"n_pg<={n_max}")[["n_pg","Re(freq)"]].values
    except:
        vert_freqs = ts.query(f"l_obs=={degree}").query(f"n_obs>={n_min}").query(f"n_obs<={n_max}")[["n_obs","f_obs"]].values
    if len(vert_freqs>0):
        slope, intercept, r_value, p_value, std_err = linregress(vert_freqs[:,0], vert_freqs[:,1])
    else:
        slope, intercept, r_value, p_value, std_err = np.zeros(5)
    return len(vert_freqs), slope, intercept, r_value, p_value, std_err

def model_epsilon(ts):
    """
    Calls the fit_radial function to determine the epsilon value for a star's pulsations.
    
    Input: Theoretical (or observed) spectrum in pandas df format.
    Output: Epsilon
    """
    length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    eps = intercept/slope
    if length_rad < 3:
        length_dip, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=1)
        if length_dip > length_rad:
            eps = intercept/slope - 0.5 # take the ell=1 values and subtract 0.5 to equal epsilon (ell=0)
    return np.round(eps, 3)

def model_Dnu(ts):
    """
    Calls the fit_radial function to determine the Delta nu value for a star's pulsations.
    
    Input: Theoretical (or observed) spectrum in pandas df format.
    Output: Delta nu
    """
    length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    if length_rad < 3:
        length_dip, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=1)
        if length_rad > length_dip:
            # redo radial
            length_rad, slope, intercept, r_value, p_value, std_err = fit_radial(ts, degree=0)
    Dnu = slope
    return np.round(Dnu, 3)

def get_fit(l, freq):
    Dnu = model_Dnu(freq)
    epsilon = model_epsilon(freq)
    return Dnu, epsilon

In [ ]:
data_dict = {}

k = 0
with progress.Progress() as pbar:
    task = pbar.add_task("[green]Processing...", total=len(hists))
    for hist, freq, n_prof, prof_index in zip(hists, freqs, n_profs, prof_indexes):
        # print(k)
        key = k
        models = [i[0] for i in prof_index if i[2] in n_prof]
        data_dict[key] = {}
        age_i = []
        cno_i = []
        pp_i = []
        eps_i = []
        dnu_i = []
        L_nuc_i = []
        density_i = []
        for i in range(len(freq)):
            try:
                Dnu, epsilon = get_fit(0, freq[i])
                eps_i.append(epsilon)
                age_i.append(hist[hist.model_number == models[i]]['star_age']/1e6)
                dnu_i.append(Dnu)
                cno_i.append(hist[hist.model_number == models[i]]['cno'].values[0])
                pp_i.append(hist[hist.model_number == models[i]]['pp'].values[0])
                L_nuc_i.append(hist[hist.model_number == models[i]]['log_Lnuc'].values[0])
                density_i.append(hist[hist.model_number == models[i]]['log_cntr_Rho'].values[0])
            except:
                pass
        data_dict[key]['age'] = age_i
        data_dict[key]['cno'] = cno_i
        data_dict[key]['pp'] = pp_i
        data_dict[key]['L_nuc'] = L_nuc_i
        data_dict[key]['dnu'] = dnu_i
        data_dict[key]['eps'] = eps_i
        data_dict[key]['density'] = density_i
        k += 1
        pbar.advance(task)

In [ ]:
data_dict[0]['age']

In [8]:
hists[0]

,model_number,num_zones,star_age,log_dt,star_mass,log_LH,log_LHe,log_LZ,log_Lnuc,pp,...,center_h1,center_he4,average_h1,average_he4,total_mass_h1,total_mass_he4,delta_nu,nu_max,num_retries,num_iters
0,1,609,1.000000e-05,-5.000000,1.2,4.639183,-99.000000,-99.000000,4.639183,4.639183,...,0.737489,0.261489,0.737489,0.261489,0.884987,0.313786,22.625170,320.474767,0,5
1,2,761,1.150000e-05,-5.823909,1.2,4.638885,-99.000000,-99.000000,4.638885,4.638885,...,0.737489,0.261489,0.737489,0.261489,0.884987,0.313786,22.623646,320.456384,3,16
2,3,842,1.168750e-05,-6.726999,1.2,4.638860,-99.000000,-99.000000,4.638860,4.638860,...,0.737489,0.261489,0.737489,0.261489,0.884987,0.313786,22.623527,320.454728,6,17
3,4,842,1.187500e-05,-6.726999,1.2,4.638845,-99.000000,-99.000000,4.638845,4.638845,...,0.737489,0.261489,0.737489,0.261489,0.884987,0.313786,22.623497,320.454675,6,10
4,5,842,1.210000e-05,-6.647817,1.2,4.638831,-99.000000,-99.000000,4.638831,4.638831,...,0.737489,0.261489,0.737489,0.261489,0.884987,0.313786,22.623468,320.454725,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,1123,771,1.510066e+09,7.302755,1.2,0.752351,-36.659144,-6.767057,0.752351,0.723967,...,0.432108,0.566923,0.680933,0.317699,0.817120,0.381239,118.613111,2520.277531,6,10
375,1124,771,1.528095e+09,7.255972,1.2,0.755250,-36.571806,-6.737358,0.755250,0.724655,...,0.426346,0.572689,0.680096,0.318536,0.816115,0.382243,118.269698,2509.408553,6,10
376,1125,771,1.543796e+09,7.195915,1.2,0.757201,-36.508848,-6.715887,0.757201,0.725021,...,0.422166,0.576869,0.679363,0.319268,0.815236,0.383121,117.883509,2498.198107,6,10
377,1126,770,1.562636e+09,7.275096,1.2,0.759781,-36.426103,-6.688402,0.759781,0.725352,...,0.416440,0.582598,0.678478,0.320152,0.814174,0.384182,117.443935,2485.074553,6,10
